# Chapter 5: 実践的応用例

## 学習目標
- ベイズ線形回帰をMCMCで実装する
- ロジスティック回帰のベイズ推定を学ぶ
- 階層モデルの構築と推定を理解する
- 状態空間モデルへの応用を習得する
- 変化点検出問題を解く
- 実際のデータを用いた分析を実践する

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.special import expit  # ロジスティック関数
from scipy.optimize import minimize
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification, make_regression
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_style("whitegrid")
np.random.seed(42)

## 5.1 ベイズ線形回帰

線形回帰モデルに対するベイズ推論をMCMCで実装します。

### モデル設定
$$y_i = \mathbf{x}_i^T \boldsymbol{\beta} + \epsilon_i, \quad \epsilon_i \sim N(0, \sigma^2)$$

### 事前分布
- $\boldsymbol{\beta} \sim N(\mathbf{0}, \tau^2 \mathbf{I})$
- $\sigma^2 \sim \text{InvGamma}(a, b)$

In [ ]:
# データ生成
def generate_regression_data(n_samples=100, n_features=3, noise_std=0.5, seed=42):
    """
    回帰データの生成
    """
    np.random.seed(seed)
    
    # 真の回帰係数
    true_beta = np.array([2.0, -1.5, 0.8, 1.2])  # intercept + 3 features
    
    # 説明変数
    X = np.random.randn(n_samples, n_features)
    X = np.column_stack([np.ones(n_samples), X])  # intercept項を追加
    
    # 目的変数
    y = X @ true_beta + np.random.normal(0, noise_std, n_samples)
    
    return X, y, true_beta, noise_std**2

# データ生成
X, y, true_beta, true_sigma2 = generate_regression_data()
n_samples, n_features = X.shape

print(f"データサイズ: {n_samples} x {n_features}")
print(f"真の回帰係数: {true_beta}")
print(f"真の誤差分散: {true_sigma2:.3f}")

# データの可視化
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 目的変数の分布
axes[0].hist(y, bins=20, alpha=0.7, density=True)
axes[0].set_title('Distribution of y')
axes[0].set_xlabel('y')
axes[0].set_ylabel('Density')

# 説明変数と目的変数の関係（最初の説明変数のみ）
axes[1].scatter(X[:, 1], y, alpha=0.6)
axes[1].set_title('y vs X1')
axes[1].set_xlabel('X1')
axes[1].set_ylabel('y')

# 相関行列
corr_matrix = np.corrcoef(X[:, 1:].T, y)
im = axes[2].imshow(corr_matrix, cmap='coolwarm', vmin=-1, vmax=1)
axes[2].set_title('Correlation Matrix')
axes[2].set_xticks(range(n_features))
axes[2].set_yticks(range(n_features))
axes[2].set_xticklabels(['X1', 'X2', 'X3', 'y'])
axes[2].set_yticklabels(['X1', 'X2', 'X3', 'y'])
plt.colorbar(im, ax=axes[2])

plt.tight_layout()
plt.show()

In [ ]:
def bayesian_linear_regression_gibbs(X, y, n_iterations=5000, 
                                   prior_beta_var=10.0,
                                   prior_sigma_shape=1.0, prior_sigma_rate=1.0):
    """
    ベイズ線形回帰のギブスサンプリング
    
    Parameters:
    - X: 説明変数行列 (n x p)
    - y: 目的変数 (n,)
    - n_iterations: イテレーション数
    - prior_beta_var: 回帰係数の事前分散
    - prior_sigma_shape, prior_sigma_rate: 誤差分散の事前分布パラメータ
    
    Returns:
    - beta_samples: 回帰係数のサンプル (n_iterations x p)
    - sigma2_samples: 誤差分散のサンプル (n_iterations,)
    """
    n, p = X.shape
    
    # 事前分布のパラメータ
    prior_beta_precision = np.eye(p) / prior_beta_var
    
    # 初期値
    beta = np.zeros(p)
    sigma2 = 1.0
    
    # サンプル保存用
    beta_samples = np.zeros((n_iterations, p))
    sigma2_samples = np.zeros(n_iterations)
    
    # 事前計算（効率化のため）
    XtX = X.T @ X
    Xty = X.T @ y
    
    for i in range(n_iterations):
        # Step 1: beta | sigma2, y のサンプリング
        # 事後分布は多変量正規分布
        posterior_precision = XtX / sigma2 + prior_beta_precision
        posterior_cov = np.linalg.inv(posterior_precision)
        posterior_mean = posterior_cov @ (Xty / sigma2)
        
        beta = np.random.multivariate_normal(posterior_mean, posterior_cov)
        
        # Step 2: sigma2 | beta, y のサンプリング
        # 事後分布は逆ガンマ分布
        residuals = y - X @ beta
        posterior_shape = prior_sigma_shape + n / 2
        posterior_rate = prior_sigma_rate + np.sum(residuals**2) / 2
        
        # 逆ガンマからのサンプリング（ガンマの逆数）
        sigma2 = 1 / np.random.gamma(posterior_shape, 1 / posterior_rate)
        
        # サンプル保存
        beta_samples[i] = beta
        sigma2_samples[i] = sigma2
        
        if (i + 1) % 1000 == 0:
            print(f"Iteration {i+1}/{n_iterations}")
    
    return beta_samples, sigma2_samples

# ベイズ線形回帰の実行
print("ベイズ線形回帰のギブスサンプリング実行中...")
beta_samples, sigma2_samples = bayesian_linear_regression_gibbs(X, y)

# 結果の統計
burnin = 1000
beta_post_mean = np.mean(beta_samples[burnin:], axis=0)
beta_post_std = np.std(beta_samples[burnin:], axis=0)
sigma2_post_mean = np.mean(sigma2_samples[burnin:])
sigma2_post_std = np.std(sigma2_samples[burnin:])

print(f"\n=== ベイズ推定結果 ===")
print(f"{'Parameter':<10} {'True':<8} {'Post Mean':<12} {'Post Std':<10} {'95% CI':<20}")
print("-" * 65)

for i in range(len(true_beta)):
    param_samples = beta_samples[burnin:, i]
    ci_lower = np.percentile(param_samples, 2.5)
    ci_upper = np.percentile(param_samples, 97.5)
    
    param_name = 'Intercept' if i == 0 else f'Beta_{i}'
    print(f"{param_name:<10} {true_beta[i]:<8.3f} {beta_post_mean[i]:<12.3f} "
          f"{beta_post_std[i]:<10.3f} [{ci_lower:.3f}, {ci_upper:.3f}]")

sigma2_ci_lower = np.percentile(sigma2_samples[burnin:], 2.5)
sigma2_ci_upper = np.percentile(sigma2_samples[burnin:], 97.5)
print(f"{'Sigma2':<10} {true_sigma2:<8.3f} {sigma2_post_mean:<12.3f} "
      f"{sigma2_post_std:<10.3f} [{sigma2_ci_lower:.3f}, {sigma2_ci_upper:.3f}]")

In [ ]:
# ベイズ線形回帰結果の可視化
def plot_bayesian_regression_results(beta_samples, sigma2_samples, true_beta, true_sigma2, 
                                    X, y, burnin=1000):
    """
    ベイズ線形回帰結果の包括的可視化
    """
    n_params = beta_samples.shape[1]
    fig, axes = plt.subplots(3, 4, figsize=(16, 12))
    
    # パラメータのトレースプロット
    for i in range(min(4, n_params)):
        axes[0, i].plot(beta_samples[:, i], alpha=0.8, linewidth=0.8)
        axes[0, i].axhline(true_beta[i], color='red', linestyle='--', linewidth=2)
        axes[0, i].axvline(burnin, color='gray', linestyle=':', alpha=0.7)
        param_name = 'Intercept' if i == 0 else f'Beta_{i}'
        axes[0, i].set_title(f'{param_name} Trace')
        axes[0, i].set_xlabel('Iteration')
        axes[0, i].set_ylabel('Value')
        axes[0, i].grid(True, alpha=0.3)
    
    # パラメータの事後分布
    for i in range(min(4, n_params)):
        param_samples = beta_samples[burnin:, i]
        axes[1, i].hist(param_samples, bins=50, density=True, alpha=0.7, color='skyblue')
        axes[1, i].axvline(true_beta[i], color='red', linestyle='--', linewidth=2, label='True')
        axes[1, i].axvline(np.mean(param_samples), color='blue', linestyle='-', linewidth=2, label='Posterior mean')
        
        param_name = 'Intercept' if i == 0 else f'Beta_{i}'
        axes[1, i].set_title(f'{param_name} Posterior')
        axes[1, i].set_xlabel('Value')
        axes[1, i].set_ylabel('Density')
        axes[1, i].legend()
    
    # 誤差分散の結果
    # トレースプロット
    axes[2, 0].plot(sigma2_samples, alpha=0.8, linewidth=0.8, color='green')
    axes[2, 0].axhline(true_sigma2, color='red', linestyle='--', linewidth=2)
    axes[2, 0].axvline(burnin, color='gray', linestyle=':', alpha=0.7)
    axes[2, 0].set_title('Sigma2 Trace')
    axes[2, 0].set_xlabel('Iteration')
    axes[2, 0].set_ylabel('Sigma2')
    axes[2, 0].grid(True, alpha=0.3)
    
    # 事後分布
    sigma2_post = sigma2_samples[burnin:]
    axes[2, 1].hist(sigma2_post, bins=50, density=True, alpha=0.7, color='lightgreen')
    axes[2, 1].axvline(true_sigma2, color='red', linestyle='--', linewidth=2, label='True')
    axes[2, 1].axvline(np.mean(sigma2_post), color='green', linestyle='-', linewidth=2, label='Posterior mean')
    axes[2, 1].set_title('Sigma2 Posterior')
    axes[2, 1].set_xlabel('Sigma2')
    axes[2, 1].set_ylabel('Density')
    axes[2, 1].legend()
    
    # 予測区間の可視化（最初の説明変数について）
    if X.shape[1] > 1:  # intercept以外の変数がある場合
        x_pred_range = np.linspace(X[:, 1].min(), X[:, 1].max(), 100)
        
        # 他の変数は平均値に固定
        X_pred = np.zeros((len(x_pred_range), X.shape[1]))
        X_pred[:, 0] = 1  # intercept
        X_pred[:, 1] = x_pred_range  # 対象変数
        for j in range(2, X.shape[1]):
            X_pred[:, j] = np.mean(X[:, j])  # 他の変数は平均値
        
        # 事後サンプルから予測分布を計算
        n_pred_samples = min(500, len(beta_samples[burnin:]))
        pred_samples = np.zeros((n_pred_samples, len(x_pred_range)))
        
        for i in range(n_pred_samples):
            idx = burnin + i
            beta_i = beta_samples[idx]
            sigma2_i = sigma2_samples[idx]
            
            # 平均の予測
            mu_pred = X_pred @ beta_i
            # 予測分布（観測ノイズを含む）
            pred_samples[i] = np.random.normal(mu_pred, np.sqrt(sigma2_i))
        
        # 予測区間の計算
        pred_mean = np.mean(pred_samples, axis=0)
        pred_lower = np.percentile(pred_samples, 2.5, axis=0)
        pred_upper = np.percentile(pred_samples, 97.5, axis=0)
        
        # プロット
        axes[2, 2].scatter(X[:, 1], y, alpha=0.6, s=20, label='Data')
        axes[2, 2].plot(x_pred_range, pred_mean, 'b-', linewidth=2, label='Posterior mean')
        axes[2, 2].fill_between(x_pred_range, pred_lower, pred_upper, alpha=0.3, color='blue', label='95% Prediction interval')
        
        # 真の関係
        true_pred = X_pred @ true_beta
        axes[2, 2].plot(x_pred_range, true_pred, 'r--', linewidth=2, label='True relationship')
        
        axes[2, 2].set_title('Prediction with Uncertainty')
        axes[2, 2].set_xlabel('X1')
        axes[2, 2].set_ylabel('y')
        axes[2, 2].legend()
        axes[2, 2].grid(True, alpha=0.3)
    
    # 残差分析
    # 事後平均を使った予測
    beta_mean = np.mean(beta_samples[burnin:], axis=0)
    y_pred = X @ beta_mean
    residuals = y - y_pred
    
    axes[2, 3].scatter(y_pred, residuals, alpha=0.6)
    axes[2, 3].axhline(0, color='red', linestyle='--')
    axes[2, 3].set_title('Residuals vs Fitted')
    axes[2, 3].set_xlabel('Fitted values')
    axes[2, 3].set_ylabel('Residuals')
    axes[2, 3].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# 結果の可視化
plot_bayesian_regression_results(beta_samples, sigma2_samples, true_beta, true_sigma2, X, y)

## 5.2 ベイズロジスティック回帰

二値分類問題に対するベイズ推論を実装します。

### モデル設定
$$P(y_i = 1 | \mathbf{x}_i) = \text{logit}^{-1}(\mathbf{x}_i^T \boldsymbol{\beta})$$

ロジスティック回帰では解析的な共役事前分布がないため、メトロポリス・ヘイスティングス法を使用します。

In [ ]:
# ロジスティック回帰データの生成
def generate_logistic_data(n_samples=200, n_features=2, seed=42):
    """
    ロジスティック回帰データの生成
    """
    np.random.seed(seed)
    
    # 真の回帰係数
    true_beta = np.array([0.5, 2.0, -1.5])  # intercept + 2 features
    
    # 説明変数
    X = np.random.randn(n_samples, n_features)
    X = np.column_stack([np.ones(n_samples), X])  # intercept項を追加
    
    # ロジット変換
    logits = X @ true_beta
    probabilities = expit(logits)  # ロジスティック関数
    
    # 二値目的変数
    y = np.random.binomial(1, probabilities)
    
    return X, y, true_beta, probabilities

# データ生成
X_log, y_log, true_beta_log, true_probs = generate_logistic_data()
n_samples_log, n_features_log = X_log.shape

print(f"データサイズ: {n_samples_log} x {n_features_log}")
print(f"真の回帰係数: {true_beta_log}")
print(f"正例の割合: {np.mean(y_log):.3f}")

# データの可視化
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# クラス別の散布図
class_0 = y_log == 0
class_1 = y_log == 1

axes[0].scatter(X_log[class_0, 1], X_log[class_0, 2], alpha=0.7, c='red', label='Class 0')
axes[0].scatter(X_log[class_1, 1], X_log[class_1, 2], alpha=0.7, c='blue', label='Class 1')
axes[0].set_title('Data Distribution by Class')
axes[0].set_xlabel('X1')
axes[0].set_ylabel('X2')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 真の確率 vs 説明変数
axes[1].scatter(X_log[:, 1], true_probs, c=y_log, cmap='RdYlBu', alpha=0.7)
axes[1].set_title('True Probabilities vs X1')
axes[1].set_xlabel('X1')
axes[1].set_ylabel('P(y=1)')
axes[1].grid(True, alpha=0.3)

# 目的変数の分布
axes[2].bar([0, 1], [np.sum(y_log == 0), np.sum(y_log == 1)], alpha=0.7, color=['red', 'blue'])
axes[2].set_title('Class Distribution')
axes[2].set_xlabel('Class')
axes[2].set_ylabel('Count')
axes[2].set_xticks([0, 1])

plt.tight_layout()
plt.show()

In [ ]:
def bayesian_logistic_regression_mh(X, y, n_iterations=10000, 
                                   prior_beta_var=10.0, step_size=0.1):
    """
    ベイズロジスティック回帰のメトロポリス・ヘイスティングス法
    
    Parameters:
    - X: 説明変数行列 (n x p)
    - y: 二値目的変数 (n,)
    - n_iterations: イテレーション数
    - prior_beta_var: 回帰係数の事前分散
    - step_size: 提案分布のステップサイズ
    
    Returns:
    - beta_samples: 回帰係数のサンプル (n_iterations x p)
    - acceptance_rate: 受理率
    """
    n, p = X.shape
    
    def log_likelihood(beta):
        """対数尤度関数"""
        logits = X @ beta
        # 数値安定性のための計算
        log_lik = np.sum(y * logits - np.log(1 + np.exp(np.clip(logits, -500, 500))))
        return log_lik
    
    def log_prior(beta):
        """対数事前分布"""
        return -0.5 * np.sum(beta**2) / prior_beta_var
    
    def log_posterior(beta):
        """対数事後分布"""
        return log_likelihood(beta) + log_prior(beta)
    
    # 初期値（最尤推定から開始）
    try:
        from scipy.optimize import minimize
        
        def neg_log_lik(beta):
            return -log_likelihood(beta)
        
        result = minimize(neg_log_lik, np.zeros(p), method='BFGS')
        beta_current = result.x
    except:
        beta_current = np.zeros(p)
    
    # サンプル保存用
    beta_samples = np.zeros((n_iterations, p))
    n_accepted = 0
    
    # 提案分布の共分散行列（適応的に調整）
    proposal_cov = step_size * np.eye(p)
    
    current_log_posterior = log_posterior(beta_current)
    
    for i in range(n_iterations):
        # 提案
        beta_proposed = np.random.multivariate_normal(beta_current, proposal_cov)
        proposed_log_posterior = log_posterior(beta_proposed)
        
        # 受理確率
        log_alpha = proposed_log_posterior - current_log_posterior
        alpha = min(1.0, np.exp(log_alpha))
        
        # 受理/棄却
        if np.random.rand() < alpha:
            beta_current = beta_proposed
            current_log_posterior = proposed_log_posterior
            n_accepted += 1
        
        beta_samples[i] = beta_current
        
        # 適応的ステップサイズ調整（簡易版）
        if i > 0 and i % 500 == 0:
            recent_acceptance = n_accepted / i
            if recent_acceptance < 0.2:  # 受理率が低すぎる
                proposal_cov *= 0.9
            elif recent_acceptance > 0.6:  # 受理率が高すぎる
                proposal_cov *= 1.1
        
        if (i + 1) % 2000 == 0:
            print(f"Iteration {i+1}/{n_iterations}, Acceptance rate: {n_accepted/(i+1):.3f}")
    
    acceptance_rate = n_accepted / n_iterations
    return beta_samples, acceptance_rate

# ベイズロジスティック回帰の実行
print("ベイズロジスティック回帰のMCMCサンプリング実行中...")
beta_samples_log, acceptance_rate_log = bayesian_logistic_regression_mh(X_log, y_log)

print(f"\n最終受理率: {acceptance_rate_log:.3f}")

# 結果の統計
burnin_log = 2000
beta_post_mean_log = np.mean(beta_samples_log[burnin_log:], axis=0)
beta_post_std_log = np.std(beta_samples_log[burnin_log:], axis=0)

print(f"\n=== ベイズロジスティック回帰結果 ===")
print(f"{'Parameter':<10} {'True':<8} {'Post Mean':<12} {'Post Std':<10} {'95% CI':<20}")
print("-" * 65)

for i in range(len(true_beta_log)):
    param_samples = beta_samples_log[burnin_log:, i]
    ci_lower = np.percentile(param_samples, 2.5)
    ci_upper = np.percentile(param_samples, 97.5)
    
    param_name = 'Intercept' if i == 0 else f'Beta_{i}'
    print(f"{param_name:<10} {true_beta_log[i]:<8.3f} {beta_post_mean_log[i]:<12.3f} "
          f"{beta_post_std_log[i]:<10.3f} [{ci_lower:.3f}, {ci_upper:.3f}]")

In [ ]:
# ロジスティック回帰結果の可視化
def plot_logistic_regression_results(beta_samples, X, y, true_beta, burnin=2000):
    """
    ベイズロジスティック回帰結果の可視化
    """
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    n_params = beta_samples.shape[1]
    
    # パラメータのトレースプロット
    for i in range(min(3, n_params)):
        axes[0, i].plot(beta_samples[:, i], alpha=0.8, linewidth=0.8)
        axes[0, i].axhline(true_beta[i], color='red', linestyle='--', linewidth=2)
        axes[0, i].axvline(burnin, color='gray', linestyle=':', alpha=0.7)
        param_name = 'Intercept' if i == 0 else f'Beta_{i}'
        axes[0, i].set_title(f'{param_name} Trace')
        axes[0, i].set_xlabel('Iteration')
        axes[0, i].set_ylabel('Value')
        axes[0, i].grid(True, alpha=0.3)
    
    # パラメータの事後分布
    for i in range(min(3, n_params)):
        param_samples = beta_samples[burnin:, i]
        axes[1, i].hist(param_samples, bins=50, density=True, alpha=0.7, color='lightcoral')
        axes[1, i].axvline(true_beta[i], color='red', linestyle='--', linewidth=2, label='True')
        axes[1, i].axvline(np.mean(param_samples), color='darkred', linestyle='-', linewidth=2, label='Posterior mean')
        
        param_name = 'Intercept' if i == 0 else f'Beta_{i}'
        axes[1, i].set_title(f'{param_name} Posterior')
        axes[1, i].set_xlabel('Value')
        axes[1, i].set_ylabel('Density')
        axes[1, i].legend()
    
    plt.tight_layout()
    plt.show()
    
    # 決定境界の可視化（2次元の場合）
    if X.shape[1] == 3:  # intercept + 2 features
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        
        # 事後平均による決定境界
        beta_mean = np.mean(beta_samples[burnin:], axis=0)
        
        # グリッドの作成
        x1_min, x1_max = X[:, 1].min() - 1, X[:, 1].max() + 1
        x2_min, x2_max = X[:, 2].min() - 1, X[:, 2].max() + 1
        xx1, xx2 = np.meshgrid(np.linspace(x1_min, x1_max, 100),
                               np.linspace(x2_min, x2_max, 100))
        
        grid_points = np.c_[np.ones(xx1.ravel().shape[0]), xx1.ravel(), xx2.ravel()]
        
        # 事後平均による予測確率
        prob_mean = expit(grid_points @ beta_mean).reshape(xx1.shape)
        
        # 等高線プロット
        class_0 = y == 0
        class_1 = y == 1
        
        axes[0].contourf(xx1, xx2, prob_mean, levels=50, alpha=0.6, cmap='RdYlBu')
        axes[0].contour(xx1, xx2, prob_mean, levels=[0.5], colors='black', linewidths=2)
        axes[0].scatter(X[class_0, 1], X[class_0, 2], c='red', alpha=0.7, label='Class 0')
        axes[0].scatter(X[class_1, 1], X[class_1, 2], c='blue', alpha=0.7, label='Class 1')
        axes[0].set_title('Decision Boundary (Posterior Mean)')
        axes[0].set_xlabel('X1')
        axes[0].set_ylabel('X2')
        axes[0].legend()
        
        # 不確実性の可視化
        # 複数の事後サンプルから予測確率の不確実性を計算
        n_uncertainty_samples = min(100, len(beta_samples[burnin:]))
        prob_samples = np.zeros((n_uncertainty_samples, xx1.size))
        
        for i in range(n_uncertainty_samples):
            beta_i = beta_samples[burnin + i]
            prob_samples[i] = expit(grid_points @ beta_i)
        
        prob_std = np.std(prob_samples, axis=0).reshape(xx1.shape)
        
        im = axes[1].contourf(xx1, xx2, prob_std, levels=20, cmap='Reds')
        axes[1].scatter(X[class_0, 1], X[class_0, 2], c='white', edgecolors='black', alpha=0.7, label='Class 0')
        axes[1].scatter(X[class_1, 1], X[class_1, 2], c='black', alpha=0.7, label='Class 1')
        axes[1].set_title('Prediction Uncertainty (Std Dev)')
        axes[1].set_xlabel('X1')
        axes[1].set_ylabel('X2')
        axes[1].legend()
        plt.colorbar(im, ax=axes[1])
        
        plt.tight_layout()
        plt.show()

# 結果の可視化
plot_logistic_regression_results(beta_samples_log, X_log, y_log, true_beta_log)

# 予測性能の評価
burnin_log = 2000
beta_mean = np.mean(beta_samples_log[burnin_log:], axis=0)
prob_pred = expit(X_log @ beta_mean)
y_pred = (prob_pred > 0.5).astype(int)

accuracy = np.mean(y_pred == y_log)
print(f"\n予測精度: {accuracy:.3f}")

# 混同行列
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_log, y_pred)
print(f"\n混同行列:")
print(cm)
print(f"\n分類レポート:")
print(classification_report(y_log, y_pred))

## 5.3 階層モデル（Hierarchical Model）

異なるグループ間で共通の構造を持つモデルを考えます。学校別の学生成績データを例として使用します。

### モデル設定
- レベル1（個人レベル）: $y_{ij} \sim N(\mu_j, \sigma^2)$
- レベル2（グループレベル）: $\mu_j \sim N(\alpha, \tau^2)$

ここで、$i$は個人、$j$はグループを表します。

In [ ]:
# 階層データの生成
def generate_hierarchical_data(n_groups=8, n_per_group=15, seed=42):
    """
    階層構造データの生成（学校別学生成績の例）
    """
    np.random.seed(seed)
    
    # 超パラメータ
    alpha_true = 75.0  # 全体平均
    tau_true = 8.0     # グループ間のばらつき
    sigma_true = 5.0   # 個人内のばらつき
    
    # グループ平均
    group_means = np.random.normal(alpha_true, tau_true, n_groups)
    
    # 個人データ
    data = []
    group_ids = []
    
    for j in range(n_groups):
        group_data = np.random.normal(group_means[j], sigma_true, n_per_group)
        data.extend(group_data)
        group_ids.extend([j] * n_per_group)
    
    return np.array(data), np.array(group_ids), group_means, alpha_true, tau_true, sigma_true

# データ生成
y_hier, group_ids, true_group_means, true_alpha, true_tau, true_sigma = generate_hierarchical_data()
n_groups = len(np.unique(group_ids))
n_total = len(y_hier)

print(f"データサイズ: {n_total} observations, {n_groups} groups")
print(f"真の超パラメータ: α = {true_alpha:.1f}, τ = {true_tau:.1f}, σ = {true_sigma:.1f}")
print(f"真のグループ平均: {true_group_means.round(1)}")

# データの可視化
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# グループ別ボックスプロット
group_data = [y_hier[group_ids == j] for j in range(n_groups)]
axes[0].boxplot(group_data, labels=[f'Group {j+1}' for j in range(n_groups)])
axes[0].scatter(range(1, n_groups+1), true_group_means, color='red', s=50, label='True means')
axes[0].axhline(true_alpha, color='red', linestyle='--', alpha=0.7, label='Overall mean')
axes[0].set_title('Data by Group')
axes[0].set_xlabel('Group')
axes[0].set_ylabel('Score')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 全体分布
axes[1].hist(y_hier, bins=30, alpha=0.7, density=True, color='skyblue')
axes[1].axvline(true_alpha, color='red', linestyle='--', linewidth=2, label='True overall mean')
axes[1].axvline(np.mean(y_hier), color='blue', linestyle='-', linewidth=2, label='Sample mean')
axes[1].set_title('Overall Distribution')
axes[1].set_xlabel('Score')
axes[1].set_ylabel('Density')
axes[1].legend()

# グループ平均の分布
sample_group_means = [np.mean(y_hier[group_ids == j]) for j in range(n_groups)]
axes[2].scatter(range(1, n_groups+1), sample_group_means, color='blue', s=50, label='Sample means')
axes[2].scatter(range(1, n_groups+1), true_group_means, color='red', s=50, label='True means')
axes[2].axhline(true_alpha, color='red', linestyle='--', alpha=0.7, label='Overall mean')
axes[2].set_title('Group Means Comparison')
axes[2].set_xlabel('Group')
axes[2].set_ylabel('Mean Score')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# グループ別統計
print(f"\n=== グループ別統計 ===")
print(f"{'Group':<8} {'N':<5} {'Sample Mean':<12} {'True Mean':<10} {'Sample Std':<12}")
print("-" * 55)
for j in range(n_groups):
    group_data_j = y_hier[group_ids == j]
    print(f"{j+1:<8} {len(group_data_j):<5} {np.mean(group_data_j):<12.2f} "
          f"{true_group_means[j]:<10.2f} {np.std(group_data_j, ddof=1):<12.2f}")

In [ ]:
def hierarchical_model_gibbs(y, group_ids, n_iterations=8000,
                           prior_alpha_mean=70, prior_alpha_var=100,
                           prior_tau_shape=1, prior_tau_rate=1,
                           prior_sigma_shape=1, prior_sigma_rate=1):
    """
    階層モデルのギブスサンプリング
    
    Parameters:
    - y: 観測値
    - group_ids: グループID
    - n_iterations: イテレーション数
    - prior_*: 事前分布のパラメータ
    
    Returns:
    - alpha_samples: 全体平均のサンプル
    - tau2_samples: グループ間分散のサンプル
    - sigma2_samples: 個人内分散のサンプル
    - mu_samples: グループ平均のサンプル
    """
    n_groups = len(np.unique(group_ids))
    n_total = len(y)
    
    # グループごとのデータ
    group_data = [y[group_ids == j] for j in range(n_groups)]
    group_sizes = [len(group_data[j]) for j in range(n_groups)]
    
    # 初期値
    alpha = np.mean(y)
    tau2 = 1.0
    sigma2 = 1.0
    mu = np.array([np.mean(group_data[j]) for j in range(n_groups)])
    
    # サンプル保存用
    alpha_samples = np.zeros(n_iterations)
    tau2_samples = np.zeros(n_iterations)
    sigma2_samples = np.zeros(n_iterations)
    mu_samples = np.zeros((n_iterations, n_groups))
    
    for i in range(n_iterations):
        # Step 1: μ_j | α, τ², σ², y のサンプリング
        for j in range(n_groups):
            n_j = group_sizes[j]
            y_j_bar = np.mean(group_data[j])
            
            # 事後分布のパラメータ
            posterior_precision = 1/tau2 + n_j/sigma2
            posterior_var = 1 / posterior_precision
            posterior_mean = posterior_var * (alpha/tau2 + n_j*y_j_bar/sigma2)
            
            mu[j] = np.random.normal(posterior_mean, np.sqrt(posterior_var))
        
        # Step 2: α | μ, τ² のサンプリング
        posterior_precision_alpha = 1/prior_alpha_var + n_groups/tau2
        posterior_var_alpha = 1 / posterior_precision_alpha
        posterior_mean_alpha = posterior_var_alpha * (prior_alpha_mean/prior_alpha_var + np.sum(mu)/tau2)
        
        alpha = np.random.normal(posterior_mean_alpha, np.sqrt(posterior_var_alpha))
        
        # Step 3: τ² | α, μ のサンプリング
        posterior_shape_tau = prior_tau_shape + n_groups / 2
        posterior_rate_tau = prior_tau_rate + np.sum((mu - alpha)**2) / 2
        
        tau2 = 1 / np.random.gamma(posterior_shape_tau, 1/posterior_rate_tau)
        
        # Step 4: σ² | μ, y のサンプリング
        total_ss = 0
        for j in range(n_groups):
            total_ss += np.sum((group_data[j] - mu[j])**2)
        
        posterior_shape_sigma = prior_sigma_shape + n_total / 2
        posterior_rate_sigma = prior_sigma_rate + total_ss / 2
        
        sigma2 = 1 / np.random.gamma(posterior_shape_sigma, 1/posterior_rate_sigma)
        
        # サンプル保存
        alpha_samples[i] = alpha
        tau2_samples[i] = tau2
        sigma2_samples[i] = sigma2
        mu_samples[i] = mu
        
        if (i + 1) % 2000 == 0:
            print(f"Iteration {i+1}/{n_iterations}")
    
    return alpha_samples, tau2_samples, sigma2_samples, mu_samples

# 階層モデルの実行
print("階層モデルのギブスサンプリング実行中...")
alpha_samples, tau2_samples, sigma2_samples, mu_samples = hierarchical_model_gibbs(y_hier, group_ids)

# 結果の統計
burnin_hier = 2000

alpha_post_mean = np.mean(alpha_samples[burnin_hier:])
tau_post_mean = np.sqrt(np.mean(tau2_samples[burnin_hier:]))
sigma_post_mean = np.sqrt(np.mean(sigma2_samples[burnin_hier:]))
mu_post_mean = np.mean(mu_samples[burnin_hier:], axis=0)

print(f"\n=== 階層モデル推定結果 ===")
print(f"{'Parameter':<15} {'True':<8} {'Post Mean':<12} {'95% CI':<20}")
print("-" * 60)

# 超パラメータ
alpha_ci = np.percentile(alpha_samples[burnin_hier:], [2.5, 97.5])
print(f"{'Alpha (mean)':<15} {true_alpha:<8.2f} {alpha_post_mean:<12.2f} "
      f"[{alpha_ci[0]:.2f}, {alpha_ci[1]:.2f}]")

tau_ci = np.percentile(np.sqrt(tau2_samples[burnin_hier:]), [2.5, 97.5])
print(f"{'Tau (group SD)':<15} {true_tau:<8.2f} {tau_post_mean:<12.2f} "
      f"[{tau_ci[0]:.2f}, {tau_ci[1]:.2f}]")

sigma_ci = np.percentile(np.sqrt(sigma2_samples[burnin_hier:]), [2.5, 97.5])
print(f"{'Sigma (ind SD)':<15} {true_sigma:<8.2f} {sigma_post_mean:<12.2f} "
      f"[{sigma_ci[0]:.2f}, {sigma_ci[1]:.2f}]")

print("\n=== グループ平均の推定 ===")
print(f"{'Group':<8} {'True':<8} {'Post Mean':<12} {'95% CI':<20}")
print("-" * 50)

for j in range(n_groups):
    mu_j_ci = np.percentile(mu_samples[burnin_hier:, j], [2.5, 97.5])
    print(f"{j+1:<8} {true_group_means[j]:<8.2f} {mu_post_mean[j]:<12.2f} "
          f"[{mu_j_ci[0]:.2f}, {mu_j_ci[1]:.2f}]")

In [ ]:
# 階層モデル結果の可視化
def plot_hierarchical_results(alpha_samples, tau2_samples, sigma2_samples, mu_samples,
                            y, group_ids, true_alpha, true_tau, true_sigma, true_group_means,
                            burnin=2000):
    """
    階層モデル結果の包括的可視化
    """
    n_groups = mu_samples.shape[1]
    
    fig, axes = plt.subplots(3, 4, figsize=(16, 12))
    
    # 超パラメータのトレースプロット
    axes[0, 0].plot(alpha_samples, alpha=0.8, linewidth=0.8)
    axes[0, 0].axhline(true_alpha, color='red', linestyle='--', linewidth=2)
    axes[0, 0].axvline(burnin, color='gray', linestyle=':', alpha=0.7)
    axes[0, 0].set_title('Alpha (Overall Mean)')
    axes[0, 0].set_xlabel('Iteration')
    axes[0, 0].set_ylabel('Alpha')
    axes[0, 0].grid(True, alpha=0.3)
    
    axes[0, 1].plot(np.sqrt(tau2_samples), alpha=0.8, linewidth=0.8, color='green')
    axes[0, 1].axhline(true_tau, color='red', linestyle='--', linewidth=2)
    axes[0, 1].axvline(burnin, color='gray', linestyle=':', alpha=0.7)
    axes[0, 1].set_title('Tau (Between-group SD)')
    axes[0, 1].set_xlabel('Iteration')
    axes[0, 1].set_ylabel('Tau')
    axes[0, 1].grid(True, alpha=0.3)
    
    axes[0, 2].plot(np.sqrt(sigma2_samples), alpha=0.8, linewidth=0.8, color='orange')
    axes[0, 2].axhline(true_sigma, color='red', linestyle='--', linewidth=2)
    axes[0, 2].axvline(burnin, color='gray', linestyle=':', alpha=0.7)
    axes[0, 2].set_title('Sigma (Within-group SD)')
    axes[0, 2].set_xlabel('Iteration')
    axes[0, 2].set_ylabel('Sigma')
    axes[0, 2].grid(True, alpha=0.3)
    
    # 分散成分の比較
    tau_post = np.sqrt(tau2_samples[burnin:])
    sigma_post = np.sqrt(sigma2_samples[burnin:])
    icc_post = tau2_samples[burnin:] / (tau2_samples[burnin:] + sigma2_samples[burnin:])
    
    axes[0, 3].hist(icc_post, bins=50, alpha=0.7, density=True, color='purple')
    true_icc = true_tau**2 / (true_tau**2 + true_sigma**2)
    axes[0, 3].axvline(true_icc, color='red', linestyle='--', linewidth=2, label='True ICC')
    axes[0, 3].axvline(np.mean(icc_post), color='purple', linestyle='-', linewidth=2, label='Post mean')
    axes[0, 3].set_title('Intraclass Correlation (ICC)')
    axes[0, 3].set_xlabel('ICC')
    axes[0, 3].set_ylabel('Density')
    axes[0, 3].legend()
    
    # 超パラメータの事後分布
    axes[1, 0].hist(alpha_samples[burnin:], bins=50, alpha=0.7, density=True, color='skyblue')
    axes[1, 0].axvline(true_alpha, color='red', linestyle='--', linewidth=2, label='True')
    axes[1, 0].axvline(np.mean(alpha_samples[burnin:]), color='blue', linestyle='-', linewidth=2, label='Post mean')
    axes[1, 0].set_title('Alpha Posterior')
    axes[1, 0].set_xlabel('Alpha')
    axes[1, 0].set_ylabel('Density')
    axes[1, 0].legend()
    
    axes[1, 1].hist(tau_post, bins=50, alpha=0.7, density=True, color='lightgreen')
    axes[1, 1].axvline(true_tau, color='red', linestyle='--', linewidth=2, label='True')
    axes[1, 1].axvline(np.mean(tau_post), color='green', linestyle='-', linewidth=2, label='Post mean')
    axes[1, 1].set_title('Tau Posterior')
    axes[1, 1].set_xlabel('Tau')
    axes[1, 1].set_ylabel('Density')
    axes[1, 1].legend()
    
    axes[1, 2].hist(sigma_post, bins=50, alpha=0.7, density=True, color='lightsalmon')
    axes[1, 2].axvline(true_sigma, color='red', linestyle='--', linewidth=2, label='True')
    axes[1, 2].axvline(np.mean(sigma_post), color='orange', linestyle='-', linewidth=2, label='Post mean')
    axes[1, 2].set_title('Sigma Posterior')
    axes[1, 2].set_xlabel('Sigma')
    axes[1, 2].set_ylabel('Density')
    axes[1, 2].legend()
    
    # グループ平均の収束（最初の4グループ）
    colors = plt.cm.tab10(np.linspace(0, 1, min(4, n_groups)))
    for j in range(min(4, n_groups)):
        axes[1, 3].plot(mu_samples[:, j], alpha=0.7, color=colors[j], 
                       linewidth=0.8, label=f'Group {j+1}')
        axes[1, 3].axhline(true_group_means[j], color=colors[j], linestyle='--', alpha=0.7)
    axes[1, 3].axvline(burnin, color='gray', linestyle=':', alpha=0.7)
    axes[1, 3].set_title('Group Means Convergence')
    axes[1, 3].set_xlabel('Iteration')
    axes[1, 3].set_ylabel('Group Mean')
    axes[1, 3].legend()
    axes[1, 3].grid(True, alpha=0.3)
    
    # 縮小効果の可視化
    sample_group_means = np.array([np.mean(y[group_ids == j]) for j in range(n_groups)])
    posterior_group_means = np.mean(mu_samples[burnin:], axis=0)
    
    axes[2, 0].scatter(sample_group_means, posterior_group_means, alpha=0.7, s=50)
    axes[2, 0].plot([sample_group_means.min(), sample_group_means.max()], 
                   [sample_group_means.min(), sample_group_means.max()], 
                   'r--', alpha=0.7, label='No shrinkage')
    axes[2, 0].set_title('Shrinkage Effect')
    axes[2, 0].set_xlabel('Sample Group Mean')
    axes[2, 0].set_ylabel('Posterior Group Mean')
    axes[2, 0].legend()
    axes[2, 0].grid(True, alpha=0.3)
    
    # グループ平均の比較
    x_pos = np.arange(n_groups)
    width = 0.25
    
    axes[2, 1].bar(x_pos - width, true_group_means, width, label='True', alpha=0.7, color='red')
    axes[2, 1].bar(x_pos, sample_group_means, width, label='Sample', alpha=0.7, color='blue')
    axes[2, 1].bar(x_pos + width, posterior_group_means, width, label='Posterior', alpha=0.7, color='green')
    
    # 信頼区間の追加
    for j in range(n_groups):
        ci = np.percentile(mu_samples[burnin:, j], [2.5, 97.5])
        axes[2, 1].errorbar(j + width, posterior_group_means[j], 
                           yerr=[[posterior_group_means[j] - ci[0]], [ci[1] - posterior_group_means[j]]], 
                           fmt='none', color='black', capsize=3)
    
    axes[2, 1].set_title('Group Means Comparison')
    axes[2, 1].set_xlabel('Group')
    axes[2, 1].set_ylabel('Mean')
    axes[2, 1].set_xticks(x_pos)
    axes[2, 1].set_xticklabels([f'G{j+1}' for j in range(n_groups)])
    axes[2, 1].legend()
    axes[2, 1].grid(True, alpha=0.3)
    
    # 予測分布
    # 新しいグループの予測分布
    new_group_samples = np.random.normal(alpha_samples[burnin:], np.sqrt(tau2_samples[burnin:]))
    
    axes[2, 2].hist(new_group_samples, bins=50, alpha=0.7, density=True, color='lightcyan', label='New group prediction')
    axes[2, 2].axvline(true_alpha, color='red', linestyle='--', linewidth=2, label='True overall mean')
    axes[2, 2].set_title('Prediction for New Group')
    axes[2, 2].set_xlabel('Predicted Group Mean')
    axes[2, 2].set_ylabel('Density')
    axes[2, 2].legend()
    
    # 個人レベル予測
    # 既存グループの新しい個人の予測（グループ1の例）
    group_0_individual_pred = np.random.normal(mu_samples[burnin:, 0], np.sqrt(sigma2_samples[burnin:]))
    
    axes[2, 3].hist(group_0_individual_pred, bins=50, alpha=0.7, density=True, color='lightpink', 
                   label='New individual in Group 1')
    axes[2, 3].axvline(true_group_means[0], color='red', linestyle='--', linewidth=2, label='True Group 1 mean')
    axes[2, 3].set_title('Individual Prediction (Group 1)')
    axes[2, 3].set_xlabel('Predicted Score')
    axes[2, 3].set_ylabel('Density')
    axes[2, 3].legend()
    
    plt.tight_layout()
    plt.show()

# 結果の可視化
plot_hierarchical_results(alpha_samples, tau2_samples, sigma2_samples, mu_samples,
                         y_hier, group_ids, true_alpha, true_tau, true_sigma, true_group_means)

# 縮小効果の定量化
burnin_hier = 2000
sample_group_means = np.array([np.mean(y_hier[group_ids == j]) for j in range(n_groups)])
posterior_group_means = np.mean(mu_samples[burnin_hier:], axis=0)
overall_mean = np.mean(y_hier)

# 縮小率の計算
shrinkage_factors = 1 - (posterior_group_means - overall_mean) / (sample_group_means - overall_mean)

print(f"\n=== 縮小効果 ===")
print(f"{'Group':<8} {'Sample Mean':<12} {'Posterior Mean':<15} {'Shrinkage':<10}")
print("-" * 50)
for j in range(n_groups):
    print(f"{j+1:<8} {sample_group_means[j]:<12.2f} {posterior_group_means[j]:<15.2f} {shrinkage_factors[j]:<10.3f}")

print(f"\n平均縮小率: {np.mean(np.abs(shrinkage_factors)):.3f}")

## 5.4 演習問題

### 問題1：変化点検出
時系列データにおいて平均が変化する点を検出するモデルを実装してください。

In [ ]:
# 問題1: 変化点検出モデル
def generate_changepoint_data(n=100, changepoint=50, mu1=0, mu2=3, sigma=1, seed=42):
    """
    変化点のある時系列データを生成
    """
    np.random.seed(seed)
    
    y = np.zeros(n)
    y[:changepoint] = np.random.normal(mu1, sigma, changepoint)
    y[changepoint:] = np.random.normal(mu2, sigma, n - changepoint)
    
    return y, changepoint

# データ生成
y_cp, true_cp = generate_changepoint_data()

# 可視化
plt.figure(figsize=(10, 6))
plt.plot(y_cp, 'b-', alpha=0.7, linewidth=1)
plt.axvline(true_cp, color='red', linestyle='--', linewidth=2, label=f'True changepoint: {true_cp}')
plt.title('Time Series with Changepoint')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# ここに変化点検出のMCMCモデルを実装してください
# ヒント：
# 1. 変化点の位置をパラメータとして扱う
# 2. 変化点前後の平均を別々にモデル化
# 3. 変化点の事前分布を設定（一様分布など）

def changepoint_detection_mcmc(y, n_iterations=5000):
    """
    変化点検出のMCMC実装
    
    モデル:
    y_t ~ N(mu1, sigma^2) for t < tau
    y_t ~ N(mu2, sigma^2) for t >= tau
    
    tau ~ Uniform(1, n-1)
    mu1, mu2 ~ N(0, 100)
    sigma^2 ~ InvGamma(1, 1)
    """
    # ここに実装してください
    pass  # 学習者が実装

# テスト
# cp_samples, mu1_samples, mu2_samples, sigma2_samples = changepoint_detection_mcmc(y_cp)
# # 結果の分析と可視化

### 問題2：ポアソン回帰
カウントデータに対するベイズポアソン回帰を実装してください。

In [ ]:
# 問題2: ベイズポアソン回帰
def generate_poisson_data(n_samples=150, n_features=2, seed=42):
    """
    ポアソン回帰データの生成
    """
    np.random.seed(seed)
    
    # 真の回帰係数
    true_beta = np.array([1.0, 0.5, -0.3])  # intercept + 2 features
    
    # 説明変数
    X = np.random.randn(n_samples, n_features)
    X = np.column_stack([np.ones(n_samples), X])  # intercept項を追加
    
    # リンク関数（対数リンク）
    log_lambda = X @ true_beta
    lambda_param = np.exp(log_lambda)
    
    # ポアソン分布からサンプリング
    y = np.random.poisson(lambda_param)
    
    return X, y, true_beta

# データ生成
X_pois, y_pois, true_beta_pois = generate_poisson_data()

# データの可視化
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# カウントデータの分布
axes[0].hist(y_pois, bins=range(max(y_pois)+2), alpha=0.7, density=False)
axes[0].set_title('Count Data Distribution')
axes[0].set_xlabel('Count')
axes[0].set_ylabel('Frequency')

# 説明変数との関係
axes[1].scatter(X_pois[:, 1], y_pois, alpha=0.6)
axes[1].set_title('Count vs X1')
axes[1].set_xlabel('X1')
axes[1].set_ylabel('Count')

axes[2].scatter(X_pois[:, 2], y_pois, alpha=0.6)
axes[2].set_title('Count vs X2')
axes[2].set_xlabel('X2')
axes[2].set_ylabel('Count')

plt.tight_layout()
plt.show()

print(f"データサイズ: {X_pois.shape[0]} x {X_pois.shape[1]}")
print(f"真の回帰係数: {true_beta_pois}")
print(f"カウントデータの平均: {np.mean(y_pois):.2f}")
print(f"カウントデータの分散: {np.var(y_pois):.2f}")

# ここにベイズポアソン回帰を実装してください
# ヒント：
# 1. 対数尤度: Σ[y_i * log(λ_i) - λ_i - log(y_i!)]
# 2. λ_i = exp(X_i @ β)
# 3. MHサンプリングを使用（解析的共役がないため）

def bayesian_poisson_regression_mh(X, y, n_iterations=8000, prior_beta_var=10.0, step_size=0.1):
    """
    ベイズポアソン回帰のメトロポリス・ヘイスティングス法
    
    モデル:
    y_i ~ Poisson(λ_i)
    log(λ_i) = X_i @ β
    β_j ~ N(0, prior_beta_var)
    """
    # ここに実装してください
    pass  # 学習者が実装

# テスト
# beta_samples_pois, acceptance_rate_pois = bayesian_poisson_regression_mh(X_pois, y_pois)
# # 結果の分析と可視化

## まとめ

この章では、MCMCを用いた実践的なベイズ推論の応用例を学習しました：

### 実装したモデル

1. **ベイズ線形回帰**：
   - ギブスサンプリングによる効率的な実装
   - 予測区間の不確実性定量化
   - 回帰診断とモデル検証

2. **ベイズロジスティック回帰**：
   - MHサンプリングによる非共役モデル
   - 決定境界の可視化
   - 予測不確実性の評価

3. **階層モデル**：
   - 多レベルデータ構造の処理
   - 縮小推定（shrinkage estimation）の理解
   - グループ間・グループ内変動の分解

### 重要な概念

- **共役性**：ギブス vs MHサンプリングの使い分け
- **階層構造**：情報の借用（information borrowing）
- **予測分布**：新しいデータの不確実性定量化
- **モデル診断**：残差分析と適合度評価

### 実践的テクニック

- **数値安定性**：オーバーフロー/アンダーフローの回避
- **適応的調整**：ステップサイズの自動調整
- **効率的実装**：事前計算による高速化
- **可視化**：結果の解釈と診断

### モデル選択指針

| データタイプ | モデル | サンプリング手法 |
|-------------|--------|------------------|
| 連続値（正規誤差） | 線形回帰 | ギブスサンプリング |
| 二値 | ロジスティック回帰 | MHサンプリング |
| カウント | ポアソン回帰 | MHサンプリング |
| 階層構造 | 階層モデル | ギブスサンプリング |
| 時系列（変化点） | 状態空間モデル | MHサンプリング |

次の章では、より高度なMCMC手法（HMC、NUTS、変分推論など）を学習し、複雑なモデルへの対応方法を習得します。